# Soluciones - Guía de Programación Entera

Este notebook lo pueden meter en google colab y sale andando.

In [1]:
%pip install picos numpy swiglpk ipykernel --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import picos
import numpy as np

## Ejercicio 1: Asignación de Tareas

In [3]:
P = picos.Problem()

# Variables de decision: x[i,j] = 1 si persona i hace tarea j, 0 sino
# i: 0=Lola, 1=Manu
# j: 0=limpieza, 1=cocina, 2=lavado platos, 3=lavanderia
x = picos.BinaryVariable('x', (2, 4))

# Tiempos para cada persona y tarea
tiempos = np.array([
    [4.5, 7.8, 3.6, 2.9],  # Lola
    [4.9, 7.2, 4.3, 3.1]   # Manu
])

# Funcion objetivo: minimizar tiempo total
P.set_objective('min', picos.sum(tiempos[i,j] * x[i,j] for i in range(2) for j in range(4)))

# Restricciones
# Cada persona hace exactamente 2 tareas
for i in range(2):
    P.add_constraint(picos.sum(x[i,j] for j in range(4)) == 2)

# Cada tarea es hecha por exactamente 1 persona
for j in range(4):
    P.add_constraint(picos.sum(x[i,j] for i in range(2)) == 1)

P.solve(solver='glpk')

print("Asignacion optima:")
personas = ['Lola', 'Manu']
tareas = ['Limpieza', 'Cocina', 'Lavado de platos', 'Lavanderia']

for i in range(2):
    print(f"{personas[i]}:")
    for j in range(4):
        if x[i,j].value > 0.5:
            print(f"  - {tareas[j]}: {tiempos[i,j]} horas")

print(f"\nTiempo total: {P.value:.1f} horas")

Asignacion optima:
Lola:
  - Limpieza: 4.5 horas
  - Lavado de platos: 3.6 horas
Manu:
  - Cocina: 7.2 horas
  - Lavanderia: 3.1 horas

Tiempo total: 18.4 horas


## Ejercicio 2: Selección de Empleados

In [4]:
P = picos.Problem()

# Variables de decision: x[i] = 1 si se contrata candidato i, 0 sino
x = picos.BinaryVariable('x', 5)

# Datos
productividad = [8, 6, 9, 7, 8]
salario = [45, 35, 55, 40, 50]  # en miles

# Funcion objetivo: maximizar productividad total
P.set_objective('max', picos.sum(productividad[i] * x[i] for i in range(5)))

# Restricciones
# Contratar exactamente 3 empleados
P.add_constraint(picos.sum(x[i] for i in range(5)) == 3)

# Presupuesto maximo de $150k
P.add_constraint(picos.sum(salario[i] * x[i] for i in range(5)) <= 150)

P.solve(solver='glpk')

print("Candidatos seleccionados:")
productividad_total = 0
costo_total = 0

for i in range(5):
    if x[i].value > 0.5:
        print(f"Candidato {i+1}: Productividad={productividad[i]}, Salario=${salario[i]}k")
        productividad_total += productividad[i]
        costo_total += salario[i]

print(f"\nProductividad total: {productividad_total}")
print(f"Costo total: ${costo_total}k")

Candidatos seleccionados:
Candidato 1: Productividad=8, Salario=$45k
Candidato 3: Productividad=9, Salario=$55k
Candidato 5: Productividad=8, Salario=$50k

Productividad total: 25
Costo total: $150k


## Ejercicio 3: Selección de Proyectos de Desarrollo

In [5]:
P = picos.Problem()

# Variables de decision: x[i] = 1 si se selecciona proyecto i, 0 sino
x = picos.BinaryVariable('x', 5)

# Datos
ganancia = [1.0, 1.8, 1.6, 0.8, 1.4]  # millones
capital = [6, 12, 10, 4, 8]  # millones

# Funcion objetivo: maximizar ganancia total
P.set_objective('max', picos.sum(ganancia[i] * x[i] for i in range(5)))

# Restriccion: capital disponible de $20 millones
P.add_constraint(picos.sum(capital[i] * x[i] for i in range(5)) <= 20)

P.solve(solver='glpk')

print("Proyectos seleccionados:")
ganancia_total = 0
capital_usado = 0

for i in range(5):
    if x[i].value > 0.5:
        print(f"Proyecto {i+1}: Ganancia=${ganancia[i]}M, Capital=${capital[i]}M")
        ganancia_total += ganancia[i]
        capital_usado += capital[i]

print(f"\nGanancia total: ${ganancia_total}M")
print(f"Capital usado: ${capital_usado}M de $20M disponibles")


Proyectos seleccionados:
Proyecto 1: Ganancia=$1.0M, Capital=$6M
Proyecto 3: Ganancia=$1.6M, Capital=$10M
Proyecto 4: Ganancia=$0.8M, Capital=$4M

Ganancia total: $3.4000000000000004M
Capital usado: $20M de $20M disponibles


## Ejercicio 4: Planificación de Horarios

In [6]:
P = picos.Problem()

# Variables de decision: x[i,j] = 1 si conductor i trabaja turno j, 0 sino
x = picos.BinaryVariable('x', (4, 6))

# Eficiencias
eficiencias = np.array([
    [8, 6, 9, 7, 8, 6],
    [7, 8, 6, 9, 7, 8],
    [9, 7, 8, 6, 9, 7],
    [6, 9, 7, 8, 6, 9]
])

# Funcion objetivo: maximizar eficiencia total
P.set_objective('max', picos.sum(eficiencias[i,j] * x[i,j] for i in range(4) for j in range(6)))

# Restricciones
# Cada conductor trabaja maximo 2 turnos
for i in range(4):
    P.add_constraint(picos.sum(x[i,j] for j in range(6)) <= 2)

# Cada turno es cubierto por exactamente 1 conductor
for j in range(6):
    P.add_constraint(picos.sum(x[i,j] for i in range(4)) == 1)

P.solve(solver='glpk')

print("Asignacion de turnos:")
for i in range(4):
    turnos_asignados = []
    eficiencia_conductor = 0
    for j in range(6):
        if x[i,j].value > 0.5:
            turnos_asignados.append(j+1)
            eficiencia_conductor += eficiencias[i,j]
    print(f"Conductor {i+1}: Turnos {turnos_asignados}, Eficiencia total: {eficiencia_conductor}")

print(f"\nEficiencia total del sistema: {P.value}")

Asignacion de turnos:
Conductor 1: Turnos [3], Eficiencia total: 9
Conductor 2: Turnos [4], Eficiencia total: 9
Conductor 3: Turnos [1, 5], Eficiencia total: 18
Conductor 4: Turnos [2, 6], Eficiencia total: 18

Eficiencia total del sistema: 54.0


## Ejercicio 5: Planificación de Turnos Hospitalarios

In [7]:
P = picos.Problem()

# Variables de decision: x[i,j] = 1 si medico i trabaja turno j, 0 sino
x = picos.BinaryVariable('x', (5, 7))

# Eficiencias (0=Mazzoleni, 1=Torcia, 2=Martinez, 3=Fargas, 4=Rodriguez)
eficiencias = np.array([
    [9, 8, 7, 0, 0, 0, 0],  # Mazzoleni
    [7, 8, 9, 8, 7, 6, 8],  # Torcia
    [6, 7, 8, 9, 8, 7, 6],  # Martinez
    [8, 7, 6, 7, 8, 9, 7],  # Fargas
    [7, 6, 8, 9, 7, 8, 9]   # Rodriguez
])

# Funcion objetivo: maximizar eficiencia total
P.set_objective('max', picos.sum(eficiencias[i,j] * x[i,j] for i in range(5) for j in range(7)))

# Restricciones
# Cada medico trabaja maximo 3 turnos
for i in range(5):
    P.add_constraint(picos.sum(x[i,j] for j in range(7)) <= 3)

# Cada turno es cubierto por exactamente 1 medico
for j in range(7):
    P.add_constraint(picos.sum(x[i,j] for i in range(5)) == 1)

# Mazzoleni solo puede trabajar turnos 1,2,3 (indices 0,1,2)
for j in range(3, 7):
    P.add_constraint(x[0,j] == 0)

# Torcia no puede trabajar turnos consecutivos
for j in range(6):
    P.add_constraint(x[1,j] + x[1,j+1] <= 1)

# Martinez debe trabajar al menos 2 turnos
P.add_constraint(picos.sum(x[2,j] for j in range(7)) >= 2)

# Fargas y Rodriguez no pueden trabajar el mismo dia
for j in range(7):
    P.add_constraint(x[3,j] + x[4,j] <= 1)

P.solve(solver='glpk')

print("Asignacion de turnos hospitalarios:")
medicos = ['Dr. Mazzoleni', 'Dra. Torcia', 'Dr. Martinez', 'Dra. Fargas', 'Dr. Rodriguez']

for i in range(5):
    turnos_asignados = []
    eficiencia_medico = 0
    for j in range(7):
        if x[i,j].value > 0.5:
            turnos_asignados.append(j+1)
            eficiencia_medico += eficiencias[i,j]
    print(f"{medicos[i]}: Turnos {turnos_asignados}, Eficiencia: {eficiencia_medico}")

print(f"\nEficiencia total: {P.value}")

Asignacion de turnos hospitalarios:
Dr. Mazzoleni: Turnos [1, 2], Eficiencia: 17
Dra. Torcia: Turnos [3], Eficiencia: 9
Dr. Martinez: Turnos [4, 5], Eficiencia: 17
Dra. Fargas: Turnos [6], Eficiencia: 9
Dr. Rodriguez: Turnos [7], Eficiencia: 9

Eficiencia total: 61.0


## Ejercicio 6: Inversiones de Capital con Restricciones

In [8]:
P = picos.Problem()

x = picos.BinaryVariable('x', 6)

# Datos
ganancia = [15, 12, 16, 18, 9, 11]
capital = [38, 33, 39, 45, 23, 27]

# Funcion objetivo
P.set_objective('max', picos.sum(ganancia[i] * x[i] for i in range(6)))

# Restricciones
P.add_constraint(picos.sum(capital[i] * x[i] for i in range(6)) <= 100)  # capital disponible
P.add_constraint(x[0] + x[1] <= 1)  # 1 y 2 mutuamente excluyentes
P.add_constraint(x[2] + x[3] <= 1)  # 3 y 4 mutuamente excluyentes
P.add_constraint(x[2] + x[3] <= x[0] + x[1])  # 3 o 4 solo si 1 o 2

P.solve(solver='glpk')

print("Inversiones seleccionadas:")
for i in range(6):
    print(f"x{i+1} = {x[i].value}")
print(f"Ganancia total = {P.value}")


Inversiones seleccionadas:
x1 = 1.0
x2 = 0.0
x3 = 1.0
x4 = 0.0
x5 = 1.0
x6 = 0.0
Ganancia total = 40.0


## Ejercicio 7: Planificación de Producción

In [9]:
P = picos.Problem()

x1 = picos.RealVariable('x1')  # barras de remolque
x2 = picos.RealVariable('x2')  # barras estabilizadoras
y1 = picos.BinaryVariable('y1')  # si se producen barras de remolque (costo fijo)
y2 = picos.BinaryVariable('y2')  # si se producen barras estabilizadoras (costo fijo)

M = 100000  # numero grande

# Funcion objetivo (ahora resta los costos fijos)
P.set_objective('max', 130*x1 + 150*x2 - 500*y1 - 600*y2)

# Restricciones
P.add_constraint(3.2*x1 + 2.4*x2 <= 16)  # maquina 1
P.add_constraint(2*x1 + 3*x2 <= 15)      # maquina 2
P.add_constraint(x1 <= M*y1)              # solo producir barras de remolque si se incurre en costo fijo
P.add_constraint(x2 <= M*y2)              # solo producir barras estabilizadoras si se incurre en costo fijo
P.add_constraint(x1 >= 0)
P.add_constraint(x2 >= 0)

P.solve(solver='glpk')

print(f"x1 = {x1.value:.2f}")
print(f"x2 = {x2.value:.2f}")
print(f"y1 = {y1.value}")
print(f"y2 = {y2.value}")
print(f"Z = {P.value:.2f}")

x1 = 5.00
x2 = 0.00
y1 = 1.0
y2 = 0.0
Z = 150.00


## Ejercicio 8: Distribución de Productos

In [10]:
P = picos.Problem()

x = picos.BinaryVariable('x', (3, 4))  # almacen i atiende cliente j

# Costos y datos
costos = np.array([
    [12, 15, 18, 20],
    [14, 12, 16, 19], 
    [16, 14, 13, 17]
])
capacidades = [200, 150, 180]
demandas = [120, 80, 100, 130]

# Funcion objetivo
P.set_objective('min', picos.sum(costos[i,j] * demandas[j] * x[i,j] for i in range(3) for j in range(4)))

# Restricciones
# Cada cliente atendido por exactamente un almacen
for j in range(4):
    P.add_constraint(picos.sum(x[i,j] for i in range(3)) == 1)

# Capacidad de cada almacen
for i in range(3):
    P.add_constraint(picos.sum(demandas[j] * x[i,j] for j in range(4)) <= capacidades[i])

P.solve(solver='glpk')

print("Solucion:")
for i in range(3):
    for j in range(4):
        print(f"x[{i+1},{j+1}] = {x[i,j].value}")
print(f"Costo total = {P.value}")

Solucion:
x[1,1] = 1.0
x[1,2] = 0.0
x[1,3] = 0.0
x[1,4] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 0.0
x[2,4] = 1.0
x[3,1] = 0.0
x[3,2] = 1.0
x[3,3] = 1.0
x[3,4] = 0.0
Costo total = 6330.0


## Ejercicio 9: Selección de Proveedores

In [8]:
import numpy as np
import picos

P = picos.Problem()

# Variables
x = picos.RealVariable('x', (3, 4))
y = picos.BinaryVariable('y', (3, 4))

# Datos
costos = np.array([
    [25, 30, 28, 32],  # A
    [40, 35, 42, 38],  # B
    [15, 18, 16, 20]   # C
])
capacidades = np.array([
    [500, 400, 600, 300],  # A
    [300, 500, 400, 600],  # B
    [800, 600, 700, 500]   # C
])
demandas = np.array([400, 350, 600])

# M grande 1000000
M = 1e6

# Objetivo
P.set_objective('min', picos.sum(costos[i, j] * x[i, j] for i in range(3) for j in range(4)))

# Restricciones
# Demanda exacta por materia prima
for i in range(3):
    P.add_constraint(picos.sum(x[i, j] for j in range(4)) == demandas[i])

# Maximo 60% por proveedor
for i in range(3):
    for j in range(4):
        P.add_constraint(x[i, j] <= 0.6 * demandas[i])

# Capacidades
for i in range(3):
    for j in range(4):
        P.add_constraint(x[i, j] <= capacidades[i, j])

# Enlace uso-flujo (osea si no le compro no puedo usar de Prov i) y limite de a lo sumo 2 proveedores por materia
for i in range(3):
    P.add_constraint(picos.sum(y[i, j] for j in range(4)) <= 2)
    for j in range(4):
        P.add_constraint(x[i, j] <= M * y[i, j])

# Incompatibilidad: usar A con prov 1 incompatible con B con prov 3
P.add_constraint(y[0, 0] + y[1, 2] <= 1)

# No negatividad
P.add_constraint(x >= 0)

P.solve(solver='glpk')

print("Costo total =", float(P.value))
for i in range(3):
    for j in range(4):
        if y[i, j].value > 0.5:
            print(f"y[{i+1},{j+1}] = 1")
        if x[i, j].value > 1e-6:
            print(f"x[{i+1},{j+1}] = {x[i, j].value:.1f}")

Costo total = 32390.0
y[1,1] = 1
x[1,1] = 240.0
y[1,3] = 1
x[1,3] = 160.0
y[2,2] = 1
x[2,2] = 210.0
y[2,4] = 1
x[2,4] = 140.0
y[3,1] = 1
x[3,1] = 360.0
y[3,3] = 1
x[3,3] = 240.0


## Ejercicio 10: Planificación de Red de Distribución

In [12]:
P = picos.Problem()

y = picos.BinaryVariable('y', 4)  # centro i abierto
x = picos.BinaryVariable('x', (4, 6))  # centro i atiende cliente j

# Datos
costos_transporte = np.array([
    [15, 18, 20, 22, 25, 28],
    [12, 15, 18, 20, 23, 26],
    [18, 16, 14, 17, 20, 23],
    [20, 18, 16, 14, 17, 20]
])
capacidades = [300, 250, 200, 180]
costos_fijos = [50000, 40000, 35000, 30000]
demandas = [80, 120, 100, 90, 110, 70]

# Funcion objetivo
P.set_objective('min', 
                picos.sum(costos_fijos[i] * y[i] for i in range(4)) +
                picos.sum(costos_transporte[i,j] * demandas[j] * x[i,j] for i in range(4) for j in range(6)))

# Restricciones
P.add_constraint(picos.sum(y[i] for i in range(4)) <= 3)  # maximo 3 centros

# Cada cliente atendido por exactamente un centro
for j in range(6):
    P.add_constraint(picos.sum(x[i,j] for i in range(4)) == 1)

# Capacidad de centros
for i in range(4):
    P.add_constraint(picos.sum(demandas[j] * x[i,j] for j in range(6)) <= capacidades[i] * y[i])

# Restricciones especiales
P.add_constraint(y[1] >= y[0])  # si centro 1, entonces centro 2
P.add_constraint(y[2] + y[3] <= 1)  # centros 3 y 4 no juntos
P.add_constraint(y[3] <= y[0] + y[1])  # centro 4 solo si 1 o 2

# Solo centros abiertos pueden atender
for i in range(4):
    for j in range(6):
        P.add_constraint(x[i,j] <= y[i])

P.solve(solver='glpk')

print("Centros abiertos:")
for i in range(4):
    print(f"y{i+1} = {y[i].value}")

print("\nAsignaciones:")
for i in range(4):
    for j in range(6):
        if x[i,j].value > 0.5:
            print(f"x[{i+1},{j+1}] = {x[i,j].value}")

print(f"\nCosto total = {P.value}")

Centros abiertos:
y1 = 1.0
y2 = 1.0
y3 = 0.0
y4 = 1.0

Asignaciones:
x[1,3] = 1.0
x[1,4] = 1.0
x[2,1] = 1.0
x[2,2] = 1.0
x[4,5] = 1.0
x[4,6] = 1.0

Costo total = 130010.0


## Ejercicio 11: Producción de Juguetes

In [47]:
P = picos.Problem()

# x_ij: Cantidad del juguete i a producir en la fabrica j
x11, x21 = picos.RealVariable('x_fab1', 2) 
x12, x22 = picos.RealVariable('x_fab2', 2) 

# y_j: 1 si se usa la fabrica j
y1, y2 = picos.BinaryVariable('y', 2)

# z_i: 1 si se produce el juguete i (para los costos fijos)
z1, z2 = picos.BinaryVariable('z', 2)

M = 100000

P.set_objective('max', 10*(x11+x12) + 15*(x21+x22) - 50000*z1 - 80000*z2)

# Restricciones
P.add_constraint(y1 + y2 <= 1) # Como maximo una fabrica
P.add_constraint(x11/50 + x21/40 <= 500) # Horas disponibles Fabrica 1
P.add_constraint(x12/40 + x22/25 <= 700) # Horas disponibles Fabrica 2
P.add_constraint(x11 + x21 <= M*y1) # Solo producir en Fabrica 1 si se usa
P.add_constraint(x12 + x22 <= M*y2) # Solo producir en Fabrica 2 si se usa
P.add_constraint(x11 + x12 <= M*z1) # Activar costo fijo para Juguete 1
P.add_constraint(x21 + x22 <= M*z2) # Activar costo fijo para Juguete 2
P.add_constraint(x11 >= 0)
P.add_constraint(x21 >= 0)
P.add_constraint(x12 >= 0)
P.add_constraint(x22 >= 0) # No-negatividad

P.solve(solver='glpk')

if y1.value > 0.5:
    print(f"Elegir Fabrica 1.")
    print(f"Producir {x11.value:.0f} unidades del Juguete 1.")
    print(f"Producir {x21.value:.0f} unidades del Juguete 2.")
elif y2.value > 0.5:
    print(f"Elegir Fabrica 2.")
    print(f"Producir {x12.value:.0f} unidades del Juguete 1.")
    print(f"Producir {x22.value:.0f} unidades del Juguete 2.")
else:
    print("No producir nada.")

print(f"\nGanancia Neta: ${P.value:.2f}")


Elegir Fabrica 2.
Producir 28000 unidades del Juguete 1.
Producir 0 unidades del Juguete 2.

Ganancia Neta: $230000.00


## Ejercicio 12: Selección de Líneas de Productos

In [14]:
P = picos.Problem()

x = picos.RealVariable('x', 4)  # niveles de produccion
y = picos.BinaryVariable('y', 4)  # si se produce
z = picos.BinaryVariable('z')  # cual restriccion usar

M = 10000

# Funcion objetivo
P.set_objective('max', 70*x[0] + 60*x[1] + 90*x[2] + 80*x[3] - 
                50000*y[0] - 40000*y[1] - 70000*y[2] - 60000*y[3])

# Restricciones
# Maximo 2 productos
P.add_constraint(picos.sum(y[i] for i in range(4)) <= 2)

# Producto 3 o 4 solo si 1 o 2
P.add_constraint(y[2] + y[3] <= y[0] + y[1])

# Restricciones de capacidad (o una u otra)
P.add_constraint(5*x[0] + 3*x[1] + 6*x[2] + 4*x[3] <= 6000 + M*(1-z))
P.add_constraint(4*x[0] + 6*x[1] + 3*x[2] + 5*x[3] <= 6000 + M*z)

# Produccion solo si se selecciona
for i in range(4):
    P.add_constraint(x[i] <= M*y[i])

P.add_constraint(x >= 0)

P.solve(solver='glpk')

print("Solucion:")
for i in range(4):
    print(f"x{i+1} = {x[i].value:.2f}")
    print(f"y{i+1} = {y[i].value}")
print(f"z = {z.value}")
print(f"Ganancia = {P.value:.2f}")

Solucion:
x1 = 0.00
y1 = 0.0
x2 = 2000.00
y2 = 1.0
x3 = 0.00
y3 = 0.0
x4 = 0.00
y4 = 0.0
z = 1.0
Ganancia = 80000.00


## Ejercicio 13: Planificación de Producción Multiperíodo

In [32]:
P = picos.Problem()

x = picos.RealVariable('x', (3, 4))  # produccion producto i periodo t
I = picos.RealVariable('I', (3, 4))  # inventario producto i periodo t
y = picos.BinaryVariable('y', (3, 4))  # se produce producto i periodo t

# Datos
costo_prod = [25, 30, 35]
costo_almac = [2, 3, 4]
cap_max = [200, 150, 100]
cap_min = [10, 5, 5]  # Reduzco aun mas las capacidades minimas
demandas = np.array([
    [50, 60, 70, 80],      # producto 1 - reduzco mas la demanda
    [40, 50, 60, 50],      # producto 2
    [30, 40, 50, 60]       # producto 3
])

# Funcion objetivo
P.set_objective('min', 
                picos.sum(costo_prod[i] * x[i,t] for i in range(3) for t in range(4)) +
                picos.sum(costo_almac[i] * I[i,t] for i in range(3) for t in range(4)))

# Restricciones
# Capacidad total por periodo
for t in range(4):
    P.add_constraint(picos.sum(x[i,t] for i in range(3)) <= 400)

# Balance de inventario
for i in range(3):
    # Periodo 1
    P.add_constraint(x[i,0] - I[i,0] == demandas[i,0])
    # Periodos 2-4
    for t in range(1, 4):
        P.add_constraint(I[i,t-1] + x[i,t] - I[i,t] == demandas[i,t])

# Capacidades min/max si se produce
for i in range(3):
    for t in range(4):
        P.add_constraint(x[i,t] <= cap_max[i] * y[i,t])
        P.add_constraint(x[i,t] >= cap_min[i] * y[i,t])

# Si se produce producto 1, producto 3 maximo 30 unidades
M = 1000  # numero grande
for t in range(4):
    P.add_constraint(x[2,t] <= 30 + M*(1-y[0,t]))

# Producto 2 en al menos 2 periodos
P.add_constraint(picos.sum(y[1,t] for t in range(4)) >= 2)

# Sin inventario final
for i in range(3):
    P.add_constraint(I[i,3] == 0)

P.add_constraint(x >= 0)
P.add_constraint(I >= 0)

P.solve(solver='glpk')

print("Solucion produccion:")
for i in range(3):
    for t in range(4):
        print(f"x[{i+1},{t+1}] = {x[i,t].value:.2f}")

print("\nSolucion inventario:")
for i in range(3):
    for t in range(4):
        print(f"I[{i+1},{t+1}] = {I[i,t].value:.2f}")

print(f"\nCosto total = {P.value:.2f}")

Solucion produccion:
x[1,1] = 110.00
x[1,2] = 0.00
x[1,3] = 150.00
x[1,4] = 0.00
x[2,1] = 40.00
x[2,2] = 50.00
x[2,3] = 60.00
x[2,4] = 50.00
x[3,1] = 30.00
x[3,2] = 60.00
x[3,3] = 30.00
x[3,4] = 60.00

Solucion inventario:
I[1,1] = 60.00
I[1,2] = 0.00
I[1,3] = 80.00
I[1,4] = 0.00
I[2,1] = 0.00
I[2,2] = 0.00
I[2,3] = 0.00
I[2,4] = 0.00
I[3,1] = 0.00
I[3,2] = 20.00
I[3,3] = 0.00
I[3,4] = 0.00

Costo total = 19160.00


## Ejercicio 14: Compra de Aviones

In [33]:
P = picos.Problem()

x1 = picos.RealVariable('x1')  # aviones largo alcance
x2 = picos.RealVariable('x2')  # aviones mediano alcance
x3 = picos.RealVariable('x3')  # aviones corto alcance

# Funcion objetivo
P.set_objective('max', 4.2*x1 + 3*x2 + 2.3*x3)

# Restricciones
P.add_constraint(67*x1 + 50*x2 + 35*x3 <= 1500)  # presupuesto
P.add_constraint(x1 + x2 + x3 <= 30)  # pilotos
P.add_constraint((5/3)*x1 + (4/3)*x2 + x3 <= 40)  # mantenimiento

P.add_constraint(x1 >= 0)
P.add_constraint(x2 >= 0)
P.add_constraint(x3 >= 0)

P.solve(solver='glpk')

print(f"x1 = {x1.value:.2f}")
print(f"x2 = {x2.value:.2f}")
print(f"x3 = {x3.value:.2f}")
print(f"Ganancia = {P.value:.2f}")

x1 = 14.06
x2 = 0.00
x3 = 15.94
Ganancia = 95.72


## Ejercicio 15: Planificación del Festival

In [43]:
P = picos.Problem()

# Para evitar un problema de la libreria PICOS con 3 indices, creamos las 
# variables una por una y las guardamos en un diccionario de Python.
# La clave del diccionario es una tupla (i, j, k) que representa 
# la combinacion (banda, ciudad, dia).
# De esta forma, podemos seguir usando la notacion intuitiva x[i,j,k] 
# en el resto del codigo, pero evitamos el error.
x = {}
for i in range(3): # Para cada banda i
    for j in range(4): # Para cada ciudad j
        for k in range(3): # Para cada dia k
            # Creamos una variable binaria para esta combinacion y la guardamos
            x[i,j,k] = picos.BinaryVariable(f'x_{i}_{j}_{k}')
y = picos.BinaryVariable('y')  # mega festival (todas en misma ciudad)
w = picos.BinaryVariable('w')  # todas tocan mismo dia  
p = picos.BinaryVariable('p', (4, 3))  # penalizacion ciudad j dia k

# Datos
ingresos = [420, 250, 220]  # en miles
costos = [80, 60, 50]       # en miles
demanda = [30, 25, 20]      # en miles
capacidades = [50, 40, 35, 45]  # en miles

M = 1000

# Funcion objetivo
P.set_objective('max', 
                picos.sum((ingresos[i] - costos[i]) * picos.sum(x[i,j,k] for j in range(4) for k in range(3)) for i in range(3)) -
                30 * picos.sum(p[j,k] for j in range(4) for k in range(3)))

# Restricciones
# Cada banda toca maximo una vez
for i in range(3):
    P.add_constraint(picos.sum(x[i,j,k] for j in range(4) for k in range(3)) <= 1)

# Marki y Manusa restricciones
for k in range(3):
    # Marki solo puede tocar en un lugar por dia (esta en las 3 bandas)
    P.add_constraint(picos.sum(x[i,j,k] for i in range(3) for j in range(4)) <= 1)
    # Manusa solo puede tocar en un lugar por dia (Pajaro y Superlocro)
    P.add_constraint(picos.sum(x[i,j,k] for i in [0,2] for j in range(4)) <= 1)

# Mega festival: o todas en misma ciudad o ciudades diferentes
# Si y=1, todas las bandas en Buenos Aires (ciudad 0)
P.add_constraint(picos.sum(x[i,0,k] for i in range(3) for k in range(3)) >= 3*y)

# Mismo dia: si w=1, todas tocan viernes (dia 0)
P.add_constraint(picos.sum(x[i,j,0] for i in range(3) for j in range(4)) >= 3*w)

# Capacidad con penalizacion
for j in range(4):
    for k in range(3):
        P.add_constraint(picos.sum(demanda[i] * x[i,j,k] for i in range(3)) <= capacidades[j] + M*p[j,k])

P.solve(solver='glpk')

print("Solucion:")
for i in range(3):
    for j in range(4):
        for k in range(3):
            if x[i,j,k].value > 0.5:
                print(f"Banda {i+1} en ciudad {j+1} dia {k+1}")

print(f"y (mega festival) = {y.value}")
print(f"w (mismo dia) = {w.value}")
print(f"Ganancia = {P.value:.2f}")

Solucion:
Banda 1 en ciudad 1 dia 1
Banda 2 en ciudad 1 dia 2
Banda 3 en ciudad 1 dia 3
y (mega festival) = 0.0
w (mismo dia) = 0.0
Ganancia = 700.00
